In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [24]:
filings_url = 'https://www.sec.gov/cgi-bin/browse-edgar?'
header = {'User-Agent': 'XtraByte Consulting, Inc., bkowalczyk@xtrabyteconsulting.com',
          'Accept': 'application/json',
          'Accept-Encoding': 'gzip, deflate'}
params = {'type' : '4',
          'owner' : 'include',
          'count' : '100',
          'action' : 'getcurrent',
          'output' : 'atom'}

latest_filings = requests.get(filings_url, params=params, headers=header)

In [25]:
if latest_filings.status_code == 200:
    print("Successfully retrieved the latest filings.")

Successfully retrieved the latest filings.


In [26]:
latest_filings

<Response [200]>

In [5]:
import lxml
print(lxml.__version__)


5.3.0


In [27]:
latest_filings_soup = BeautifulSoup(latest_filings.text, 'lxml-xml')
latest_filings_soup.contents


[<feed xmlns="http://www.w3.org/2005/Atom">
 <title>Latest Filings - Thu, 19 Dec 2024 21:44:28 EST</title>
 <link href="/cgi-bin/browse-edgar?action=getcurrent" rel="alternate"/>
 <link href="/cgi-bin/browse-edgar?action=getcurrent" rel="self"/>
 <id>https://www.sec.gov/cgi-bin/browse-edgar?action=getcurrent</id>
 <author><name>Webmaster</name><email>webmaster@sec.gov</email></author>
 <updated>2024-12-19T21:44:28-05:00</updated>
 <entry>
 <title>4 - BERKSHIRE HATHAWAY INC (0001067983) (Reporting)</title>
 <link href="https://www.sec.gov/Archives/edgar/data/1067983/000095017024138717/0000950170-24-138717-index.htm" rel="alternate" type="text/html"/>
 <summary type="html">
  &lt;b&gt;Filed:&lt;/b&gt; 2024-12-19 &lt;b&gt;AccNo:&lt;/b&gt; 0000950170-24-138717 &lt;b&gt;Size:&lt;/b&gt; 30 KB
 </summary>
 <updated>2024-12-19T21:42:24-05:00</updated>
 <category label="form type" scheme="https://www.sec.gov/" term="4"/>
 <id>urn:tag:sec.gov,2008:accession-number=0000950170-24-138717</id>
 </en

In [28]:
entries = latest_filings_soup.find_all('entry')
filings = []

for entry in entries:
    filing = {}
    filing['title'] = entry.title.text
    filing['link'] = entry.link['href']
    filing['updated'] = entry.updated.text
    filings.append(filing)
    
filings_df = pd.DataFrame(filings)
filings_df.head()

,title,link,updated
0,4 - BERKSHIRE HATHAWAY INC (0001067983) (Repor...,https://www.sec.gov/Archives/edgar/data/106798...,2024-12-19T21:42:24-05:00
1,4 - VERISIGN INC/CA (0001014473) (Issuer),https://www.sec.gov/Archives/edgar/data/101447...,2024-12-19T21:42:24-05:00
2,4 - BUFFETT WARREN E (0000315090) (Reporting),https://www.sec.gov/Archives/edgar/data/315090...,2024-12-19T21:42:24-05:00
3,4 - BERKSHIRE HATHAWAY INC (0001067983) (Repor...,https://www.sec.gov/Archives/edgar/data/106798...,2024-12-19T21:39:06-05:00
4,4 - SIRIUS XM HOLDINGS INC. (0000908937) (Issuer),https://www.sec.gov/Archives/edgar/data/908937...,2024-12-19T21:39:06-05:00


In [29]:
filings

[{'title': '4 - BERKSHIRE HATHAWAY INC (0001067983) (Reporting)',
  'link': 'https://www.sec.gov/Archives/edgar/data/1067983/000095017024138717/0000950170-24-138717-index.htm',
  'updated': '2024-12-19T21:42:24-05:00'},
 {'title': '4 - VERISIGN INC/CA (0001014473) (Issuer)',
  'link': 'https://www.sec.gov/Archives/edgar/data/1014473/000095017024138717/0000950170-24-138717-index.htm',
  'updated': '2024-12-19T21:42:24-05:00'},
 {'title': '4 - BUFFETT WARREN E (0000315090) (Reporting)',
  'link': 'https://www.sec.gov/Archives/edgar/data/315090/000095017024138717/0000950170-24-138717-index.htm',
  'updated': '2024-12-19T21:42:24-05:00'},
 {'title': '4 - BERKSHIRE HATHAWAY INC (0001067983) (Reporting)',
  'link': 'https://www.sec.gov/Archives/edgar/data/1067983/000095017024138712/0000950170-24-138712-index.htm',
  'updated': '2024-12-19T21:39:06-05:00'},
 {'title': '4 - SIRIUS XM HOLDINGS INC. (0000908937) (Issuer)',
  'link': 'https://www.sec.gov/Archives/edgar/data/908937/000095017024138

In [30]:
form_urls = []
for filing in filings:
    filing_url = filing['link']
    filing_page = requests.get(filing_url, headers=header)
    filing_page_soup = BeautifulSoup(filing_page.text, 'lxml')
    filing_page_soup.title

    
    tables = filing_page_soup.find_all('table')
    for table in tables:
        #if table.caption:
        #    if table.caption.text == 'Document Format Files':
        rows = table.find_all('tr')
        i = 0
        for row in rows:
            cells = row.find_all('td')
            if i == 2:
                if cells:
                    if cells[2].a:
                        filing['file'] = cells[2].a['href']
                        print(filing['file'])
                        form_urls.append(filing['file'])
                        # break
            i += 1

/Archives/edgar/data/315090/000095017024138717/ownership.xml
/Archives/edgar/data/315090/000095017024138717/ownership.xml
/Archives/edgar/data/315090/000095017024138717/ownership.xml
/Archives/edgar/data/315090/000095017024138712/ownership.xml
/Archives/edgar/data/315090/000095017024138712/ownership.xml
/Archives/edgar/data/315090/000095017024138712/ownership.xml
/Archives/edgar/data/315090/000095017024138710/ownership.xml
/Archives/edgar/data/315090/000095017024138710/ownership.xml
/Archives/edgar/data/315090/000095017024138710/ownership.xml
/Archives/edgar/data/947033/000147450624000291/primary_doc.xml
/Archives/edgar/data/947033/000147450624000291/primary_doc.xml
/Archives/edgar/data/890394/000121390024110710/ownership.xml
/Archives/edgar/data/890394/000121390024110710/ownership.xml
/Archives/edgar/data/1391437/000151028124000734/primary_doc.xml
/Archives/edgar/data/1391437/000151028124000734/primary_doc.xml
/Archives/edgar/data/1391437/000151028124000734/primary_doc.xml
/Archives/e

In [31]:
form_urls = list(dict.fromkeys(form_urls))
print(form_urls)

['/Archives/edgar/data/315090/000095017024138717/ownership.xml', '/Archives/edgar/data/315090/000095017024138712/ownership.xml', '/Archives/edgar/data/315090/000095017024138710/ownership.xml', '/Archives/edgar/data/947033/000147450624000291/primary_doc.xml', '/Archives/edgar/data/890394/000121390024110710/ownership.xml', '/Archives/edgar/data/1391437/000151028124000734/primary_doc.xml', '/Archives/edgar/data/890394/000121390024110708/ownership.xml', '/Archives/edgar/data/1470308/000090514824003388/form4.xml', '/Archives/edgar/data/890394/000121390024110706/ownership.xml', '/Archives/edgar/data/890394/000121390024110704/ownership.xml', '/Archives/edgar/data/906709/000095017024138693/ownership.xml', '/Archives/edgar/data/1510281/000151028124000733/primary_doc.xml', '/Archives/edgar/data/1849635/000147450624000290/primary_doc.xml', '/Archives/edgar/data/93410/000112760224029836/form4.xml', '/Archives/edgar/data/93410/000112760224029835/form4.xml', '/Archives/edgar/data/1823144/00011046592

In [32]:
filing_data = []
for file_url in form_urls:
    file = requests.get('https://www.sec.gov' + file_url, headers=header)
    file_soup = BeautifulSoup(file.text, 'lxml-xml')
    symbol = file_soup.find('issuerTradingSymbol').text.strip('\n') if file_soup.find('issuerTradingSymbol') else None
    issuerCIK = file_soup.find('issuerCik').text.strip('\n') if file_soup.find('issuerCik') else None
    ownerCIK = file_soup.find('rptOwnerCik').text.strip('\n') if file_soup.find('rptOwnerCik') else None
    owner = file_soup.find('rptOwnerName').text.strip('\n') if file_soup.find('rptOwnerName') else None
    firm = file_soup.find('issuerName').text.strip('\n') if file_soup.find('issuerName') else None
    isDirector = file_soup.find('isDirector').text.strip('\n') if file_soup.find('isDirector') else None
    isOfficer = file_soup.find('isOfficer').text.strip('\n') if file_soup.find('isOfficer') else None
    isTenPercentOwner = file_soup.find('isTenPercentOwner').text.strip('\n') if file_soup.find('isTenPercentOwner') else None
    isOther = file_soup.find('isOther').text.strip('\n') if file_soup.find('isOther') else None
    officerTitle = file_soup.find('officerTitle').text.strip('\n') if file_soup.find('officerTitle') else None
    transactions = file_soup.find_all('nonDerivativeTransaction')
    for transaction in transactions:
        shares = transaction.find('transactionShares').text.strip('\n') if transaction.find('transactionShares') else None
        price = transaction.find('transactionPricePerShare').text.strip('\n') if transaction.find('transactionPricePerShare') else None
        
        file_data = {
            'Symbol': symbol,
            'Issuer CIK': issuerCIK,
            'Company': firm,
            'Owner': owner,
            'Owner CIK': ownerCIK,
            'IsDirector': isDirector,
            'IsOfficer': isOfficer,
            'IsTenPercentOwner': isTenPercentOwner,
            'IsOther': isOther,
            'Officer Title': officerTitle,
            'Transaction Date': transaction.find('transactionDate').text.strip('\n') if transaction.find('transactionDate') else None,
            'Shares': transaction.find('transactionShares').text.strip('\n') if transaction.find('transactionShares') else None,
            'Price': transaction.find('transactionPricePerShare').text.strip('\n') if transaction.find('transactionPricePerShare') else None,
            'Acquired/Disposed': transaction.find('transactionAcquiredDisposedCode').text.strip('\n') if transaction.find('transactionAcquiredDisposedCode') else None,
            'Transaction Code': transaction.find('transactionCode').text.strip('\n') if transaction.find('transactionCode') else None,
            'Equity Swap': transaction.find('equitySwapInvolved').text.strip('\n') if transaction.find('equitySwapInvolved') else None,
            'Shares Owned': transaction.find('sharesOwnedFollowingTransaction').text.strip('\n') if transaction.find('sharesOwnedFollowingTransaction') else None
        }
        filing_data.append(file_data)
    # TODO: Add logic to handle derivative transactions
filing_data_df = pd.DataFrame(filing_data)
filing_data_df.head(10)

,Symbol,Issuer CIK,Company,Owner,Owner CIK,IsDirector,IsOfficer,IsTenPercentOwner,IsOther,Officer Title,Transaction Date,Shares,Price,Acquired/Disposed,Transaction Code,Equity Swap,Shares Owned
0,VRSN,0001014473,VERISIGN INC/CA,BERKSHIRE HATHAWAY INC,0001067983,false,false,true,false,None,2024-12-17,6160,194.6302,A,P,false,12821773
1,VRSN,0001014473,VERISIGN INC/CA,BERKSHIRE HATHAWAY INC,0001067983,false,false,true,false,None,2024-12-17,76056,195.2574,A,P,false,12897829
2,VRSN,0001014473,VERISIGN INC/CA,BERKSHIRE HATHAWAY INC,0001067983,false,false,true,false,None,2024-12-17,796,196.0041,A,P,false,12898625
3,VRSN,0001014473,VERISIGN INC/CA,BERKSHIRE HATHAWAY INC,0001067983,false,false,true,false,None,2024-12-18,53503,191.474,A,P,false,12952128
4,VRSN,0001014473,VERISIGN INC/CA,BERKSHIRE HATHAWAY INC,0001067983,false,false,true,false,None,2024-12-18,2573,192.7944,A,P,false,12954701
5,VRSN,0001014473,VERISIGN INC/CA,BERKSHIRE HATHAWAY INC,0001067983,false,false,true,false,None,2024-12-18,4724,193.6495,A,P,false,12959425
6,VRSN,0001014473,VERISIGN INC/CA,BERKSHIRE HATHAWAY INC,0001067983,false,false,true,false,None,2024-12-18,19004,194.7037,A,P,false,12978429
7,VRSN,0001014473,VERISIGN INC/CA,BERKSHIRE HATHAWAY INC,0001067983,false,false,true,false,None,2024-12-18,4196,195.4786,A,P,false,12982625
8,VRSN,0001014473,VERISIGN INC/CA,BERKSHIRE HATHAWAY INC,0001067983,false,false,true,false,None,2024-12-19,1100,191.12,A,P,false,12983725
9,VRSN,0001014473,VERISIGN INC/CA,BERKSHIRE HATHAWAY INC,0001067983,false,false,true,false,None,2024-12-19,8936,192.8086,A,P,false,12992661


In [ ]:
file_data['Transaction Date'] = file_soup.find('transactionDate').text.strip('\n')
file_data['Shares'] = file_soup.find('transactionShares').text.strip('\n')
file_data['Price'] = file_soup.find('transactionPricePerShare').text.strip('\n')
file_data['Acquired/Disposed'] = file_soup.find('transactionAcquiredDisposedCode').text.strip('\n')
file_data['Transaction Code'] = file_soup.find('transactionCode').text.strip('\n')
file_data['Equity Swap'] = file_soup.find('equitySwapInvolved').text.strip('\n')
file_data['Shares Owned'] = file_soup.find('sharesOwnedFollowingTransaction').text.strip('\n')
file_data['Symbol'] = file_soup.find('issuerTradingSymbol').text.strip('\n')
file_data['Issuer CIK'] = file_soup.find('issuerCik').text.strip('\n')
file_data['Company'] = file_soup.find('issuerName').text.strip('\n')
file_data['Owner'] = file_soup.find('rptOwnerName').text.strip('\n')
file_data['Owner CIK'] = file_soup.find('rptOwnerCik').text.strip('\n')
file_data['IsDirector'] = file_soup.find('isDirector').text.strip('\n')
file_data['IsOfficer'] = file_soup.find('isOfficer').text.strip('\n')
file_data['IsTenPercentOwner'] = file_soup.find('isTenPercentOwner').text.strip('\n')
file_data['IsOther'] = file_soup.find('isOther').text.strip('\n')
file_data['Officer Title'] = file_soup.find('officerTitle').text.strip('\n')

In [ ]:
from lxml import etree

for filing in filings:
    file_url = filing['file']
    file = requests.get('https://www.sec.gov' + file_url, headers=header)
    root = etree.fromstring(file.content)
    data = []
    for element in root.iter():
        tag = etree.QName(element).localname
        value = element.text.strip() if element.text else None
        attributes = element.attrib
        data.append(data.append({
            "Tag": tag,
            "Value": value,
            "Attributes": attributes
        }))

In [17]:
all_tables = latest_filings_soup.find_all('table', summary="")
all_tables

[<table bgcolor="white" border="0" cellpadding="0" cellspacing="0" summary="" width="100%">
 <tr>
 <td rowspan="2" width="95"><a href="/index.htm">
 <img alt="U.S. Securities &amp; Exchange Commission" border="0" height="92" src="/images/bannerSeal.gif" width="95"/></a></td>
 <td colspan="2"><img alt="" height="7" src="/images/pixel.gif" width="1"/></td>
 </tr>
 <tr>
 <td width="26"><img alt="SEC Seal" border="0" height="85" src="/images/bannerSealR.gif" width="26"/></td>
 <td valign="top" width="100%">
 <table border="0" cellpadding="0" cellspacing="0" summary="" width="100%">
 <tr>
 <td bgcolor="#324395" valign="top" width="171"><img alt="" border="0" height="30" src="/images/bannerFlag.gif" width="171"/></td>
 <td align="right" bgcolor="#324395" class="gray" nowrap="nowrap" valign="middle" width="95%">
 <a href="/index.htm">Home</a> | <a href="/edgar/searchedgar/webusers.htm">EDGAR Search Home</a> |
  <a href="/cgi-bin/browse-edgar?action=getcurrent">Latest Filings</a> |
  <a href="

In [18]:
filings_table = None
for table in all_tables:
    headers = table.find_all('th')
    if headers:
        header_texts = [header.get_text(strip=True) for header in headers]
        if "Form" in header_texts and "Formats" in header_texts and "Description" in header_texts:
            filings_table = table
            print("Found the filings table.")
            print(filings_table)

Found the filings table.
<table summary="">
<tr bgcolor="#D6D6D6">
<th align="left" nowrap="nowrap" width="4%">Form</th>
<th align="left" nowrap="nowrap" width="4%">Formats</th>
<th align="left" width="40%">Description</th>
<th align="left" nowrap="nowrap" width="4%">Accepted</th>
<th align="left" nowrap="nowrap" width="4%">Filing Date</th>
<th align="left" nowrap="nowrap" width="4%">File/Film No</th>
</tr>
<tr><td>
<td>
<td align="left" bgcolor="#E6E6E6" valign="top"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0002016416&amp;owner=include&amp;count=10">Lentell Eric (0002016416) (Reporting)</a></td>
</td></td></tr>
<tr align="left" nowrap="nowrap" valign="top">
<td nowrap="nowrap">4</td>
<td nowrap="nowrap"><a href="/Archives/edgar/data/2016416/000141588924028439/0001415889-24-028439-index.htm">[html]</a><a href="/Archives/edgar/data/2016416/000141588924028439/0001415889-24-028439.txt">[text]</a></td>
<td class="small">Statement of changes in beneficial ownership of securi

In [21]:
# Check if we found the filings table
if filings_table:
    # Process the table as before
    filings = []
    for row in filings_table.find_all('tr')[1:]:  # Skip the header row
        cells = row.find_all('td')
        print('Next Row')
        print(cells)
        if len(cells) > 0:
            try:
                # Extract form type
                form_type = cells[0].get_text(strip=True)
                print('Form Type: ' + form_type)

                # Extract description
                description = cells[2].get_text(strip=True)
                print('Description: ' + description)

                # Extract filing date
                filing_date = cells[4].get_text(strip=True)
                print('Filing Date: ' + filing_date)

                # Extract filing links
                links = cells[1].find_all('a')
                form_links = {
                    link.get_text(strip=True): f"https://www.sec.gov{link['href']}" for link in links
                }
                print('Form Links: ' + str(form_links))

                # Append the filing info to the list
                filings.append({
                    "form_type": form_type,
                    "description": description,
                    "filing_date": filing_date,
                    "form_links": form_links,
                })
            except Exception as e:
                print(f"Error parsing row: {e}")

    # Output the extracted filings
    for filing in filings:
        print(filing)
else:
    print("Filings table not found.")

Next Row
[<td>
<td>
<td align="left" bgcolor="#E6E6E6" valign="top"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0002016416&amp;owner=include&amp;count=10">Lentell Eric (0002016416) (Reporting)</a></td>
</td></td>, <td>
<td align="left" bgcolor="#E6E6E6" valign="top"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0002016416&amp;owner=include&amp;count=10">Lentell Eric (0002016416) (Reporting)</a></td>
</td>, <td align="left" bgcolor="#E6E6E6" valign="top"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0002016416&amp;owner=include&amp;count=10">Lentell Eric (0002016416) (Reporting)</a></td>]
Form Type: Lentell Eric (0002016416) (Reporting)
Description: Lentell Eric (0002016416) (Reporting)
Error parsing row: list index out of range
Next Row
[<td nowrap="nowrap">4</td>, <td nowrap="nowrap"><a href="/Archives/edgar/data/2016416/000141588924028439/0001415889-24-028439-index.htm">[html]</a><a href="/Archives/edgar/data/2016416/000141588924028439/0001415889-24

In [14]:
entries = latest_filings_soup.find_all('td')
print(entries)


[<td rowspan="2" width="95"><a href="/index.htm">
<img alt="U.S. Securities &amp; Exchange Commission" border="0" height="92" src="/images/bannerSeal.gif" width="95"/></a></td>, <td colspan="2"><img alt="" height="7" src="/images/pixel.gif" width="1"/></td>, <td width="26"><img alt="SEC Seal" border="0" height="85" src="/images/bannerSealR.gif" width="26"/></td>, <td valign="top" width="100%">
<table border="0" cellpadding="0" cellspacing="0" summary="" width="100%">
<tr>
<td bgcolor="#324395" valign="top" width="171"><img alt="" border="0" height="30" src="/images/bannerFlag.gif" width="171"/></td>
<td align="right" bgcolor="#324395" class="gray" nowrap="nowrap" valign="middle" width="95%">
<a href="/index.htm">Home</a> | <a href="/edgar/searchedgar/webusers.htm">EDGAR Search Home</a> |
 <a href="/cgi-bin/browse-edgar?action=getcurrent">Latest Filings</a> |
 <a href="javascript:history.back()">Previous Page</a></td>
<td bgcolor="#324395" width="100%"><img alt="" src="/images/pixel.gif

In [ ]:
filings = []
for entry in entries:
    filing = {}
    filing['title'] = entry.find('title').text
    filing['link'] = entry.find('link')['href']
    filing['updated'] = entry.find('updated').text
    filings.append(filing)

print(filings)


In [ ]:
    
filings_df = pd.DataFrame(filings)
filings_df